### Домашнее задание <a class="anchor" id="hw"></a><center>

1. Подберите скорость обучения (eta) и количество итераций

#### Начиная с eta = 0.02 алгоритм расходится, меньшие значения, при достаточном количестве итерраций, работают. Но MSE при первых прогонах увеличивается, принимая экстремально большие значения. То есть при обучении на другом наборе данных алгоритм вполне может разойтись. С другой стороны, на этих данных при eta = 0.018, модель сходится к 120-й итеррации. В то время, как при eta = 0.012 требуется  400 прогонов, зато MSE монотонно убывает.
#### Руководствуясь данной схемой, оставил второй вариант - медленно но верно.
<img src='images/gradient_descent_lr.png' width=600px>

#### Но чтобы, таки, алгоритм быстрее сходился, изменил условие уменьшения eta, теперь оно снижается каждые 50 итерраций.

In [104]:
## n = X.shape[1]

eta = 0.0125
n_iter = 151

W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {eta} \
       \nInitial weights = {W} \n')

for i in range(n_iter):
    y_pred = np.dot(X, W)
    err = calc_mse(y, y_pred)
    for k in range(W.shape[0]):
        W[k] -= eta * (1/n * 2 * X[:, k] @ (y_pred - y))
    if i % 50 == 0: # Изменил условие уменьшения eta
        eta /= 1.1       
    if i % 10 == 0:
        print(f'Iteration #{i}: W_new = {W}, MSE = {round(err, 2)}')

Number of objects = 2        
Learning rate = 0.0125        
Initial weights = [1.  0.5] 

Iteration #0: W_new = [ 7.75   24.0625], MSE = 3047.75
Iteration #10: W_new = [23.9763241  20.28625035], MSE = 3081.57
Iteration #20: W_new = [33.45596909 17.78497311], MSE = 2565.21
Iteration #30: W_new = [38.97593422 16.05010003], MSE = 2210.48
Iteration #40: W_new = [42.17304706 14.781826  ], MSE = 1932.68
Iteration #50: W_new = [44.00856098 13.80303587], MSE = 1700.05
Iteration #60: W_new = [43.56554982  5.31106006], MSE = 84.24
Iteration #70: W_new = [44.03255683  4.15292666], MSE = 45.16
Iteration #80: W_new = [44.41352404  3.94942382], MSE = 44.2
Iteration #90: W_new = [44.66027595  3.88784667], MSE = 44.05
Iteration #100: W_new = [44.81403049  3.85785784], MSE = 44.0
Iteration #110: W_new = [44.90207238  3.84159499], MSE = 43.98
Iteration #120: W_new = [44.95893899  3.83127924], MSE = 43.97
Iteration #130: W_new = [44.99564843  3.8246225 ], MSE = 43.97
Iteration #140: W_new = [45.01934541

2*. В этом коде мы избавляемся от итераций по весам, но тут есть ошибка, исправьте ее

#### Ошибка говорит о несоответствии размерностей. Очевидно, для корректного решения нужно транспонировать матрицу Х. Кроме того, требуют корректировки eta и n_iter

In [115]:
n = X.shape[1]

# ИЗМЕНЕНИЯ
eta = 1e-3 
n_iter = 1000


W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {eta} \
       \nInitial weights = {W} \n')

for i in range(n_iter):
    y_pred = np.dot(X, W)
    err = calc_mse(y, y_pred)
    for k in range(W.shape[0]):
        W[k] -= eta * (1/n * 2 * X[:, k] @ (y_pred - y))
    # ИЗМЕНЕНИЯ
    W -= eta * (1/n * 2 * np.dot(X.T, y_pred - y))
    if i % 100 == 0:
        print(f'Iteration #{i}: W_new = {W}, MSE = {round(err,2)}')

Number of objects = 2        
Learning rate = 0.001        
Initial weights = [1.  0.5] 

Iteration #0: W_new = [2.08 4.27], MSE = 3047.75
Iteration #100: W_new = [28.38281518  6.83710367], MSE = 177.43
Iteration #200: W_new = [38.38986469  5.02247953], MSE = 65.33
Iteration #300: W_new = [42.39314129  4.29654705], MSE = 47.39
Iteration #400: W_new = [43.99463466  4.00614091], MSE = 44.52
Iteration #500: W_new = [44.63530512  3.8899652 ], MSE = 44.06
Iteration #600: W_new = [44.89160255  3.84348962], MSE = 43.98
Iteration #700: W_new = [44.99413322  3.82489726], MSE = 43.97
Iteration #800: W_new = [45.03515017  3.81745947], MSE = 43.97
Iteration #900: W_new = [45.05155882  3.81448401], MSE = 43.97


3*. Вместо того, чтобы задавать количество итераций, задайте другое условие останова алгоритма - когда веса перестают изменяться меньше определенного порога $\epsilon$.

#### Условие опасное. Алгоритм может и не сойтись, при  подборе параметров такая ситуация возможна.  Поэтому резонно оставить оба условия. 

In [141]:
n = X.shape[1]

eta = 1e-3 
n_iter = 1001
epsilon_weight_dist = 0.001
change_weight_dist = 1

W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {eta} \
       \nInitial weights = {W} \n')

while n_iter >= 0 and change_weight_dist > epsilon_weight_dist:
    y_pred = np.dot(X, W)
    err = calc_mse(y, y_pred)
    old_weight_dist = np.linalg.norm(W, ord=2)
    for k in range(W.shape[0]):
        W[k] -= eta * (1/n * 2 * X[:, k] @ (y_pred - y))
    W -= eta * (1/n * 2 * np.dot(X.T, y_pred - y))
    change_weight_dist = abs(old_weight_dist - np.linalg.norm(W, ord=2))
    n_iter -= 1
    if n_iter % 100 == 0:
        print(f'Iteration left #{n_iter}: W_new = {W}, MSE = {round(err,2)}, change weight norm {round(change_weight_dist, 4)}')



Number of objects = 2        
Learning rate = 0.001        
Initial weights = [1.  0.5] 

Iteration left #1000: W_new = [2.08 4.27], MSE = 3047.75, change weight norm 3.6316
Iteration left #900: W_new = [28.38281518  6.83710367], MSE = 177.43, change weight norm 0.1427
Iteration left #800: W_new = [38.38986469  5.02247953], MSE = 65.33, change weight norm 0.0594
Iteration left #700: W_new = [42.39314129  4.29654705], MSE = 47.39, change weight norm 0.024
Iteration left #600: W_new = [43.99463466  4.00614091], MSE = 44.52, change weight norm 0.0096
Iteration left #500: W_new = [44.63530512  3.8899652 ], MSE = 44.06, change weight norm 0.0039
Iteration left #400: W_new = [44.89160255  3.84348962], MSE = 43.98, change weight norm 0.0015
